# Pymeso : tutorial

## Initialization

### Load and initialize some modules for the interface

In [1]:
%gui qt5
import panel as pn
pn.extension()

### Load the Experiment module

This module contains the methodes used to carry an experiment, we will call it exp

In [2]:
from pymeso import Experiment
exp = Experiment()

Load some utilities for the experiment :  
- different type of sweeps 
- Alias and Amplifier

In [3]:
from pymeso.utils import LinSweep,LinSteps,LogSteps,ArraySteps
from pymeso.utils import Alias
from pymeso.instruments.utils import Ampli

### Load drivers of the connected instruments

This can be done :
- explicitely in the notebook 
- or loaded from a file (e.g. use %run config.py to execute the file config.py)

Here let's do it explicitely. We first load the generic driver for a type of instrument and then create one particular instrument with a given address.

In [4]:
from pymeso.instruments.utils import Dummy
test=Dummy()
test2=Dummy()

## Configuration of the experiment

### Define aliases for some instrument attribute
This is used to refer to intrument attributes that are often called. Here we will refer to 
- test.dac as Vbias 
- test.dac2 as Vgate 

If name is indicated it will be used as a label in the data file. Otherwise the name of the attribute is used.

In [5]:
Vbias=Alias([test,'dac'],name='Vbias')
Vgate=Alias([test,'dac2'],name='Vgate')

Get Alias value

In [6]:
Vbias()

0

Set Alias value

In [7]:
Vbias(3.14)

The old way of defining Aliases was :

In [8]:
exp.set_register('Vbias',[test,'dac'])
exp.set_register('Vgate',[test,'dac2'])
exp.register

{'Vbias': [<pymeso.instruments.utils.dummy.Dummy at 0x2100cd5ca90>, 'dac'],
 'Vgate': [<pymeso.instruments.utils.dummy.Dummy at 0x2100cd5ca90>, 'dac2']}

Prefer the new way of defining Aliases that can be used outside the Experiment module.

### Define a new instrument based on the value of another one

In [9]:
Vbias_mV=Ampli(Vbias,1000) #gain of 1000
Vgate_kV=Ampli([test,'dac2'],1e-3)  # gain of 0.001

In [10]:
Vbias(2.5)
Vbias_mV.value

2500.0

In [11]:
Vbias_mV.value=50
Vbias()

0.05

### Define the instruments that are measured in an experiment

This is define in the form of a python dictionnary of the form {'label':instrus, ...} where :
- 'label' will be the name of the corresponding data in the file
- instrus is the measured attribute of an instrument. It can have different forms :
    - 'instru' if 'instru' has been defined as an alias (see before)
    - 'instru.attribute' to measure instru.attribute
    - [instru,'attribute'] to measure instru.attribute
    - (instru,'attribute') to measure instru.attribute

In [12]:
exp.measure={'V1':Vbias,
              'wave':'test2.wave',
              'V2':[test,'dac2'],
              'wave5':'test2.wave5',
              'V3':(test2,'dac3'),
              'Time':[test,'time'],
              'wave10':[test,'wave10']}

One can get the measurement by using get_measure, and, if indicated, save it to a file.

In [13]:
exp.get_measure(format='col_multi',file='toto.dat')

,V1,wave,V2,wave5,V3,Time,wave10
0,0.05,0.683527,0,-0.991235,0,0.150753,0.974413
1,0.05,0.524700,0,-0.575696,0,0.150753,-0.303771
2,0.05,0.337336,0,0.967921,0,0.150753,0.245444
3,0.05,0.127406,0,-0.622400,0,0.150753,-0.917633
4,0.05,-0.096486,0,-0.969110,0,0.150753,0.090504
...,...,...,...,...,...,...,...
95,0.05,0.738854,0,NaN,0,0.150753,NaN
96,0.05,0.974545,0,NaN,0,0.150753,NaN
97,0.05,0.362629,0,NaN,0,0.150753,NaN
98,0.05,-0.573331,0,NaN,0,0.150753,NaN


Change values of the times used in the experiment :
- init_wait : waiting time at the beginning of a sweep, in second
- wait_time : waiting time before taking one measurement, in second

In [14]:
exp.wait_time=0.2
exp.init_wait=1

# To do the experiment

## Inline or Batch execution

The different methods of the experiment modules can be executed :
- in a Jupyter cell. In this case the instruction is executed in a different thread. It means that the user car execute other instructions during the execution of the first instruction.
- from a file (using exp.batch_file('filename')) or a multiline string (using exp.batch_line(multiline_string)). In this case the instructions are executed sequentially. The batch format has the following properties :
    - the empty lines or the lines starting by % or # are ignored
    - the method of the Experiment module can be expressed in the python way or without the python formatting (see below for some examples)  

Note that the batch is executed in a diffrent thread. The option run=False in batch_line or batch_file return the list of instructions without executing them.

## Locking mechanism

The Experiment module has a locking mechanism to prevent changing the same instrument's attribute with two different instructions. In case of problem with this locking mechanism, one can reset it by using the instruction:

In [14]:
exp.clear_lock()

## List of methods in Experiment module:
- Move
- Sweep
- Multisweep
- Record
- Megasweep
- Wait
- Get
- Set
- Spy

## Plotting the data

During or after taking the data, one can plot them with the external plotter. You can start the plotter by two means :
- execute the python file : plot_exec.py
- launch the command file : plotter.bat

# Available methods

## Move method

**Syntax:** exp.move(device, value, rate)

Used to move an instrument attribute 'device' to a given value at a given rate.  

The device can be indicated in different forms :
- 'device', if this is defined as an alias
- 'instru.attribute' to move instru.attribute
- [instru,'attribute'] to move instru.attribute
- (instru,'attribute') to move instru.attribute
- {'label':[instru,'attribute']} to move instru.attribute

In [12]:
test.dac=0
exp.move([test,'dac'],10,1)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [15]:
from threading import enumerate
enumerate()

[<_MainThread(MainThread, started 11856)>,
 <Thread(IOPub, started daemon 16748)>,
 <Heartbeat(Heartbeat, started daemon 6776)>,
 <ControlThread(Control, started daemon 16484)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 3816)>,
 <ParentPollerWindows(Thread-4, started daemon 7996)>]

In [19]:
test.dac2=0
exp.move([test,'dac2'],1,0.1)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [21]:
exp.move(Vbias,1,0.1)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [23]:
test.dac2=0
order="exp.move('test.dac2',1,0.1)"
exp.batch_line(order)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [11]:
test.dac=0
exp.move({'Vbias':[test,'dac']},1,0.1)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [ ]:
test.dac=0
exp.move('Vbias',1,0.1)

In [ ]:
test.dac=0
order="move Vbias 1 0.1"
exp.batch_line(order)

To get help you can ask for inline help 

In [ ]:
exp.move?

## Sweep method

**Syntax** : exp.sweep(device,start,end,rate,Npoints,file)  

Sweep the device defined in 'device' from 'start' to 'end' 
at a rate 'rate' with 'Npoints' points and save it to the file 'file'. 
If the  file extension is .gz, .bz2 or .xz, the file is automatically 
compressed with the corresponding algorithm.

The device can be indicated in different forms :
- 'device', if this is defined as an alias. The label in the file will be 'device'.
- 'instru.attribute' to move instru.attribute. The label in the file will be 'instru.attribute'.
- [instru,'attribute'] to move instru.attribute. The label in the file will be 'instru'.
- (instru,'attribute') to move instru.attribute. The label in the file will be 'instru'.
- {'label':[instru,'attribute']} to move instru.attribute. The label in the file will be 'label'.

**Options** :
- extra-rate : rate used outside the main loop. If None then set to rate. Default : None
- wait : if True, wait the wait_time before doing the measurement. Default : True
- back : If True, return to the start value when finished. Default : False.
- mode : define the mode of the sweep. Default : None
    * None : standard sweep
    * fly : on the fly measurement
- overwrite : If True overwrite the file, otherwise the old file is renamed. Default : False
- format : format of the data (line, line_multi, col, col_multi). Default : line
    - 'line' : tabular data are put in line with label _n for the nth element
    - 'line_multi': tabular data are put in line with a second index
    - 'col' : tabular data are put in columns with no number to fill the empty place
    - 'col_multi' : tabular data are put in columns with duplicated numbers to fill the empty place
- init_wait : value of the time waited at the beginning of the sweep, if None set to self.init_wait. Default : None
- wait_time : value of the time waited before each measurement, if None set to self.wait_time. Default : None
- measure : specify the measured quantities in the form of a python dict. if None set to self.measure. Default : None

In [17]:
#measure_dict={'V1':[test,'dac'],'wave':'test2.wave',
#              'V2':[test,'dac2'],'V3':[test,'dac3'],'Time':[test,'time']}
#exp.measure=measure_dict
exp.measure
exp.wait_time=0
exp.init_wait=1
test.dac=0
exp.sweep(Vbias,0,10,0.1,11,'test_sweep.dat',
          mode='updn',init_wait=5,extra_rate=1,format='col_multi')

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [18]:
Vbias()

-3.3999999999999995

In [13]:
exp.wait_time=0.1
exp.init_wait=1
exp.batch_line("""
exp.sweep('test.dac2',0,1,0.1,101,'test_sweep2.dat',extra_rate=1)
""")

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [17]:
exp.wait_time=0.1
exp.init_wait=1
exp.sweep ( [test,'dac2'],0, 1,0.1,101,'test_sweep1.dat', extra_rate=1)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [15]:
exp.wait_time=0.1
exp.init_wait=1
exp.sweep([test,'dac2'],0,1,0.1,11,'test_sweep1.dat',extra_rate=1,wait=True,init_wait=5,wait_time=1)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [ ]:
exp.sweep('Vbias',0,1,0.1,101,'test_sweep2.dat',format='col_multi',extra_rate=1)

In [ ]:
test.dac3=5
exp.sweep({'Vbias3':[test,'dac3']},0,1,0.01,11,'test_sweep3.dat',format='col_multi',extra_rate=0.5,overwrite=True)

In [ ]:
test.dac3=5
exp.batch_line('sweep test.dac3 0 1 0.1 11 test_sweep4.dat format=col_multi extra_rate=2 overwrite=True')

In [ ]:
test.dac=1
exp.batch_line('sweep Vbias 0 1 0.1 11 test_sweep5.dat format=col extra_rate=1 wait_time=1.2 wait=True')

You can add a comment to the file

In [19]:
comment="""This is a
multiline comment"""
exp.sweep(Vbias,0,10,1,11,'test_sweep2.dat',
          mode='updn',init_wait=5,extra_rate=1,format='col_multi',comment=comment)

Column
    [0] Column()
    [1] Column()
    [2] Column()

To get help you can ask for inline help

In [ ]:
exp.sweep?

## Multisweep method

**SYNTAX:** exp.multisweep(stepper_list,file)

Multi-sweep using a list of sweeps defined in stepper_list and save it to a file 'file'. If the file extension is .gz, .bz2 or .xz, the file is automatically compressed with the corresponding algorithm.

The stepper_list has the forms [sweep0,sweep1,...] where sweep0,sweep1,...
are sweeps of type LinSweeps,LinSteps,LogSteps,ArraySteps.

**OPTIONS:**
- overwrite : If True overwrites the file, otherwise the old file is renamed. Default : False
- format : format of the data (line, line_multi, col, col_multi). Default : line
    - 'line' : tabular data are put in line with label _n for the nth element
    - 'line_multi': tabular data are put in line with a second index
    - 'col' : tabular data are put in columns with no number to fill the empty place
    - 'col_multi' : tabular data are put in columns with duplicated numbers to fill the empty place
- measure : specify the measured quantities in the form of a python dict. if None set to self.measure. Default : None
- wait_time : value of the time waited before each mesurement, if None set to self.wait_time. Default : None
- wait : if True, wait the wait_time before doing the measurement. Default : True

In [18]:
exp.measure={'Vbias1':Vbias,
              'wave':'test2.wave',
              'V2':[test,'dac2'],
              'wave5':'test2.wave5',
              'V3':(test2,'dac3'),
              'Time':[test,'time'],
              'wave10':[test,'wave10']}
step_heater=LinSteps([test,'dac3'],0,1,5,1,name='Heater')
sweep_gate=LinSweep([test,'dac'],0,10,2,10,name='Vgate')
sweep_bias=LinSweep([test,'dac2'],-5,5,2,10,name='Vbias',mode='serpentine')
exp.multisweep([step_heater,sweep_gate,sweep_bias],'test_multisweep.dat',overwrite=True,
               comment="comment")

Column
    [0] Column()
    [1] Column()
    [2] Column()

To get help you can ask for inline help:

In [17]:
exp.multisweep?

## Record method

**Syntax :** exp.record(time_interval,npoints,file)

Record data every time_interval (in seconds) with npoints points in the file 'file'.
If the  file extension is .gz, .bz2 or .xz, the file is automatically compressed with the corresponding algorithm.

**Options** :
- overwrite : If True overwrite the file, otherwise the old file is renamed. Default : False
- format : format of the data (line, line_multi, col, col_multi). Default : line
    - 'line' : tabular data are put in line with label _n for the nth element
    - 'line_multi': tabular data are put in line with a second index
    - 'col' : tabular data are put in columns with no number to fill the empty place
    - 'col_multi' : tabular data are put in columns with duplicated numbers to fill the empty place
- measure : specify the measured quantities in the form of a python dict. if None set to self.measure. Default : None

In [23]:
measure_dict={'V1':[test,'dac']}
exp.record(1,11,'test_record1.dat',format='col_multi',measure=measure_dict)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [18]:
exp.record(1,11,'test_record1.dat',format='line_multi',overwrite=True,comment='test')

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [ ]:
exp.batch_line('record 1 11 test_record2 format=col_multi overwrite=False')

To get help you can ask for inline help:

In [ ]:
exp.record?

## Megasweep method

**Syntax:** exp.megasweep(stepper_list,file)

Multi-sweep using the list defined in stepper_list and save it to a file 'file'. If the  file extension is .gz, .bz2 or .xz, the file is automatically compressed with the corresponding algorithm.

The stepper_list has the forms [[args_1,kwargs_1],[args_2,kwargs_2],...] with :
- args_n=(device_n,start_n,end_n,rate_n,npoints_n)
- kwargs_n={'parameter1_n':value1_n,'parameter2_n':value2_n,...}

The device can be indicated in different forms :
- 'device', if this is defined as an alias. The label in the file will be 'device'.
- 'instru.attribute' to move instru.attribute. The label in the file will be 'instru.attribute'.
- [instru,'attribute'] to move instru.attribute. The label in the file will be 'instru'.
- (instru,'attribute') to move instru.attribute. The label in the file will be 'instru'.
- {'label':[instru,'attribute']} to move instru.attribute. The label in the file will be 'label'.

In a batch format one can also use a inline declaration :
'megasweep Vbias 0 1 0.5 11 init_wait=0.2 Vgate -1 1 0.5 11 init_wait=0.1 mode=updn test_megasweep_batch_updn'

**OPTIONS FOR EACH SUB-SWEEPS** :
- extra_rate : rate used outside the main loop. If None then set to rate. Default : None
- back : If True, return to the start value when finished. Default : False.
- mode : define the mode of the sweep. Default : None
    * None : standard sweep
    * serpentine : alternate forward and backwards for successive stepper
    * updn : do a forward and then a backward sweep
    * fly : on the fly measurement
- init_wait : value of the time waited at the beginning of the sweep, if None set to self.init_wait. Default : None

**GENERAL OPTIONS** :
- overwrite : If True overwrites the file, otherwise the old file is renamed. Default : False
- format : format of the data (line, line_multi, col, col_multi). Default : line
    - 'line' : tabular data are put in line with label _n for the nth element
    - 'line_multi': tabular data are put in line with a second index
    - 'col' : tabular data are put in columns with no number to fill the empty place
    - 'col_multi' : tabular data are put in columns with duplicated numbers to fill the empty place
- measure : specify the measured quantities in the form of a python dict. if None set to self.measure. Default : None
- wait_time : value of the time waited before each mesurement, if None set to self.wait_time. Default : None
- wait : if True, wait the wait_time before doing the measurement. Default : True
    
**EXAMPLES** :

stepper_list=[[[test,'dac2'],0,1,1,5),],
              [('Vgate',0,1,1,11),{'extra_rate':3,'mode':'serpentine'}]]        # if Vgate is defined in the register
exp.megasweep(stepper_list,'test_megasweep2.dat',overwrite=True)

exp.batch_line('megasweep test.dac3 0 1 0.1 11 init_wait=0.2 Vgate -1 1 0.1 11 init_wait=0.1 test_megasweep_batch format=col_multi overwrite=False wait=True wait_time=0.1')

In [ ]:
measure_dict={'V1':[test,'dac'],'wave':'test.wave',
              'V2':[test,'dac2'],'V3':[test,'dac3'],'Time':[test,'time']}
exp.wait_time=0.1
exp.init_wait=1
stepper_list=[[('test.dac2',0,10,0.1,11),{'extra_rate':10,'init_wait':1}],
             [([test,'dac3'],0,3,1,11),{'extra_rate':10,'init_wait':1,'mode':'updn'}]]
exp.megasweep(stepper_list,'test_newmegasweep_colmulti.gz',format='col_multi',
              wait=False,wait_time=0.1)

In [ ]:
exp.wait_time=0.1
exp.init_wait=0.1
test.dac=0
test.dac2=0
order='megasweep Vbias 0 1 0.5 11 init_wait=0.2 Vgate -1 1 0.5 11 init_wait=0.1 test_megasweep_batch_updn'
exp.batch_line(order)

To get help you can ask for inline help:

In [ ]:
exp.megasweep?

## Wait method

Wait during the time 'value' (in seconds) or until the condition 'value' (described by a string) is fulfilled.

In [ ]:
exp.wait(10)

In [18]:
test.dac2=0
exp.wait('test.dac2 > 2')

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [19]:
test.dac2=3

To get help you can ask for inline help:

In [ ]:
exp.wait?

## Spy method

**Syntax:** exp.spy(measure)

Launch a spy window (with a QT interface). The spied quantities are given in the measure dictionnary.
If no distionnary is provided, the self.measure dictionnary is used.

In [21]:
measure_dict={
    'V1':[test,'dac'],
    'wave':'test.wave',
    'V2':[test,'dac2'],
    'V3':[test,'dac3'],
    'Time':[test,'time']
}
exp.spy(measure_dict)

In [ ]:
exp.spy()

To get help you can ask for inline help:

In [ ]:
exp.spy?

## Get method

**Syntax:** exp.get(device)

Get the value of the device. The device can be indicated in different forms :
- 'device', if this is defined as an alias. 
- 'instru.attribute' to move instru.attribute. 
- [instru,'attribute'] to move instru.attribute.
- (instru,'attribute') to move instru.attribute.
- {'label':[instru,'attribute']} to move instru.attribute.

In [ ]:
exp.get(Vgate)

In [ ]:
exp.get('test.dac2')

In [ ]:
exp.get({'test':[test,'dac2']})

To get help you can ask for inline help:

In [ ]:
exp.get?

## Set method

**Syntax:** exp.set(device)

Set the value of the device to value. The device can be indicated in different forms :
- 'device', if this is defined as an alias. 
- 'instru.attribute' to move instru.attribute. 
- [instru,'attribute'] to move instru.attribute.
- (instru,'attribute') to move instru.attribute.
- {'label':[instru,'attribute']} to move instru.attribute.

In [ ]:
exp.set(Vgate,2.54)

In [ ]:
exp.set('test.dac2',3.14)

In [ ]:
exp.set({'test':[test,'dac2']},5.21)

To get help you can ask for inline help:

In [ ]:
exp.set?

# Examples of batch 

In [21]:
string="""
test.dac2=0
move test.dac2 1 0.1
exp.wait_time=0.1
exp.init_wait=1
exp.wait(60)
exp.sweep([test,'dac'],0,1,0.1,101,'test_sweep1.dat',extra_rate=1)
record 1 11 test_record2 format=col_multi overwrite=False
"""
exp.batch_line(string)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [ ]:
string="""
exp.set('Vbias',0)
exp.set('Vgate',0)
test.dac=3
exp.wait('test.dac < 20')
megasweep Vbias 0 1 0.5 11 init_wait=0.2 Vgate -1 1 0.5 11 init_wait=0.1 test_megasweep_batch_updn
record 1 11 test_record2 format=col_multi overwrite=False
"""
exp.batch_line(string)

In [ ]:
# define Aliases
Vbias=Alias([test,'dac'],name='Vbias')
Vgate=Alias([test,'dac2'],name='Vgate')

# define measure quantities
exp.measure={'Vbias2':Vbias,'Vgate3':[test,'dac2'],
              'Vbias2':(test2,'dac3'),'Time':[test,'time']}

# define sweeps
sweep_gate=LinSteps(Vgate,0,1,11,0.1,init_wait=1)
sweep_bias=LinSweep(Vbias,-2,2,0.5,21,mode='updn',init_wait=0.5)

# define waiting_time
exp.wait_time=0.1

# define and start batch
string="""
exp.multisweep([sweep_gate,sweep_bias],'test_multisweep.dat',overwrite=True)
exp.move('Vgate',0,1)
"""
exp.batch_line(string)

In [ ]:
{'Vbias':Vbias,'Vgate':[test,'dac2'],'Vbias':(test2,'dac3'),'Time':[test,'time']}

In [ ]:
action=tasks[0][0]

In [ ]:
action in (exp.run,)

In [ ]:
exp.multisweep([sweep_gate,sweep_bias],'test_multisweep.dat',overwrite=True)

In [ ]:
tasks[1][0](*tasks[1][1],run=False)

In [ ]:
2+2

In [ ]:
kwargs=tasks[0][2]

In [ ]:
kwargs['run']=False

In [ ]:
tasks[0][2]

In [ ]:
exp.wait(10,run=False)

# Examples of program

In [14]:
program="""
for i in range(10):
    name='data_{}'.format(i)
    exp.sweep(Vbias,0,1,0.1,11,'test_'+name)
"""
exp.program(program)

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [ ]:
i

In [2]:
from pymeso.panel_experiment_interface import Panel_Interface_Exp as Interface

In [3]:
interface=Interface()

Column
    [0] Column()
    [1] Column()
    [2] Column()

In [15]:
print('# Measure: {}'.format(exp.measure))

# Measure: {'V1': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC5E0>, 'dac'], 'wave': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC6A0>, 'wave'], 'V2': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC5E0>, 'dac2'], 'wave5': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC6A0>, 'wave5'], 'V3': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC6A0>, 'dac3'], 'Time': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC5E0>, 'time'], 'wave10': [<pymeso.instruments.utils.dummy.Dummy object at 0x00000262080DC5E0>, 'wave10']}


In [24]:
from IPython import get_ipython

In [25]:
from IPython import get_ipython

In [26]:
from IPython import get_ipython

In [27]:
shell = get_ipython().__class__.__name__

In [28]:
shell

'ZMQInteractiveShell'

In [29]:
get_ipython().run_line_magic('matplotlib', 'qt')

In [32]:
import numpy as np

In [33]:
x=np.linspace(0,6,100)
y=np.sin(x)

In [34]:
import matplotlib.pyplot as plt

In [35]:
plt.plot(x,y)

In [36]:
import pandas as pd

In [37]:
get_ipython().run_line_magic('gui', 'qt5')

In [38]:
Qconsole

NameError: name 'Qconsole' is not defined

In [39]:
%qtconsole

In [53]:
%console

UsageError: Line magic function `%console` not found.
